In [2]:
import pandas as pd
from pytrends.request import TrendReq
from tqdm import tqdm


## nasdaq 100

In [3]:
path = 'https://www.liberatedstocktrader.com/nasdaq-100-companies-list-sector-market-cap/'
data = pd.read_html(path)

In [4]:
nasdaq_sector = data[1]
nasdaq_sector

,Ticker,Description,Sector,Market Capitalization ($B)
0,AAPL,Apple Inc.,Electronic Technology,2962062
1,MSFT,Microsoft Corporation,Technology Services,2441866
2,GOOG,Alphabet Inc.,Technology Services,1717286
3,GOOGL,Alphabet Inc.,Technology Services,1717286
4,AMZN,"Amazon.com, Inc.",Retail Trade,1417160
...,...,...,...,...
96,ZM,"Zoom Video Communications, Inc.",Technology Services,21340
97,WBA,"Walgreens Boots Alliance, Inc.",Health Technology,20226
98,ENPH,"Enphase Energy, Inc.",Electronic Technology,17553
99,SIRI,Sirius XM Holdings Inc.,Consumer Services,17291


In [5]:
companies = nasdaq_sector.Description.apply(lambda x:x.replace('.', '').replace(',', '')).tolist()

In [6]:
PATH = './open/'
FILE = 'holdings-daily-us-en-xlk.xlsx'

## 기술주

In [7]:
company_df = pd.read_excel(PATH + FILE).iloc[3:72]
company_df = company_df.rename(columns=company_df.iloc[0]).iloc[1:]
company_df

,Name,Ticker,Identifier,SEDOL,Weight,Sector,Shares Held,Local Currency
4,APPLE INC,AAPL,037833100,2046251,23.230306,-,64568089,USD
5,MICROSOFT CORP,MSFT,594918104,2588173,22.836567,-,34417521,USD
6,NVIDIA CORP,NVDA,67066G104,2379504,4.514954,-,4939309,USD
7,BROADCOM INC,AVGO,11135F101,BDZ78H9,4.297453,-,2462191,USD
8,ADOBE INC,ADBE,00724F101,2008154,2.913903,-,2719457,USD
...,...,...,...,...,...,...,...,...
67,JUNIPER NETWORKS INC,JNPR,48203R104,2431846,0.111968,-,1917328,USD
68,SOLAREDGE TECHNOLOGIES INC,SEDG,83417M104,BWC52Q6,0.091831,-,337427,USD
69,DXC TECHNOLOGY CO,DXC,23355L106,BYXD7B3,0.053583,-,1224127,USD
70,US DOLLAR,-,999USDZ92,-,0.00394,-,1874970.09,USD


In [8]:
companies = list(company_df.Name)

## google trends

In [9]:
# pytrends = TrendReq(tz=540)
# kw_list = ['airbnb']
# pytrends.build_payload(kw_list)
# df = pytrends.related_queries()

In [10]:
# r = pd.concat([df[kw]['rising'] for kw in kw_list], axis=0)
# r = r.sort_values(by='value', ascending=False).reset_index(drop=True)
# r

In [11]:
def list_categories(company):
    pytrends = TrendReq(tz=540)
    kw_list = [company]
    pytrends.build_payload(kw_list)
    df = pytrends.related_queries()
    r = pd.concat([df[kw]['rising'] for kw in kw_list], axis=0)
    r = r.sort_values(by='value', ascending=False).reset_index(drop=True)
    r['company'] = company
    return r


In [12]:
list_categories('apple inc')

,query,value,company
0,moderna stock,28550,apple inc
1,meta stock,26050,apple inc
2,nio inc,23450,apple inc
3,nio inc share price,19550,apple inc
4,nio stock,16350,apple inc
5,nio inc stock,13000,apple inc
6,amd share price,10100,apple inc
7,meta platforms inc stock,7950,apple inc
8,biontech aktie,7500,apple inc
9,servicenow inc stock,7350,apple inc


In [13]:
# pd.concat([list_categories(company) for company in companies[:3]])

In [14]:
df = pd.DataFrame()
fails = []

for company in tqdm(companies):
    try:
        df = pd.concat([df, list_categories(company)])
    except:
        fails.append(company)

100%|██████████| 68/68 [02:44<00:00,  2.42s/it]


In [15]:
fails

['ACCENTURE PLC CL A',
 'INTL BUSINESS MACHINES CORP',
 'CADENCE DESIGN SYS INC',
 'AMPHENOL CORP CL A',
 'COGNIZANT TECH SOLUTIONS A',
 'CDW CORP/DE',
 'ZEBRA TECHNOLOGIES CORP CL A',
 'STATE STREET INSTITUTIONAL LIQ',
 'DXC TECHNOLOGY CO',
 'XAK TECHNOLOGY    DEC23']

In [17]:
df = df.reset_index(drop='index')
df

,query,value,company
0,moderna stock,28550,APPLE INC
1,meta stock,26050,APPLE INC
2,nio inc,23450,APPLE INC
3,nio inc share price,19550,APPLE INC
4,nio stock,16350,APPLE INC
...,...,...,...
1021,us dollar index live,300,US DOLLAR
1022,pound to pkr,250,US DOLLAR
1023,us dollar to ksh,250,US DOLLAR
1024,us dollar rate in pakistan,250,US DOLLAR


In [18]:
df.to_csv('./open/keywords.csv', index=False)